In [1]:
from net.CSRNet import CSRNet
import sys
import numpy as np
import random
import math
from eval.eval_as_whole import eval_model
from Dataset.EvalDatasetConstructor import EvalDatasetConstructor
from Dataset.TrainDatasetConstructor import TrainDatasetConstructor
from metrics import AEBatch, SEBatch
from PIL import Image
import time
import torch
# torch.backends.cudnn.benchmark=True
%matplotlib inline

In [2]:
# config
config = {
'min_mae':10240000,
'min_loss':10240000,
'eval_num':182,
'train_num':300,
'learning_rate': 1e-6,
'train_batch_size': 1,
'epoch': 20000,
'eval_per_step': 300,
'mode':'whole',
'if_random_hsi':True,
'if_flip':True,
'stage':'numeration',
'gt_map_path':"/home/zzn/Documents/Datasets/part_A_final/train_data/gt_map_w_net",
'img_path':"/home/zzn/Documents/Datasets/part_A_final/train_data/images",
'gt_map_path_t':"/home/zzn/Documents/Datasets/part_A_final/test_data/gt_map_w_net",
'img_path_t':"/home/zzn/Documents/Datasets/part_A_final/test_data/images",
'gt_path_t':"/home/zzn/Documents/Datasets/part_A_final/test_data/ground_truth",
'model_save_path':"/home/zzn/PycharmProjects/NAS_FPN/StateDicts/CSRNet_6-17_A.pkl"
}

In [3]:
# data_load
train_dataset = TrainDatasetConstructor(
    config['img_path'],
    config['gt_map_path'],
    config['train_num'],
    mode=config['mode'],
    stage=config['stage'],
    if_random_hsi=config['if_random_hsi'],
    if_flip=config['if_flip'])
eval_dataset = EvalDatasetConstructor(
    config['img_path_t'],
    config['gt_map_path_t'],
    config['eval_num'],
    mode=config['mode'],
    stage=config['stage'])
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=config['train_batch_size'])
eval_loader = torch.utils.data.DataLoader(dataset=eval_dataset, batch_size=1)

In [4]:
# obtain the gpu device
assert torch.cuda.is_available()
cuda_device = torch.device("cuda")

# model construct
net = CSRNet().cuda()
# net.load_state_dict(
#     torch.load("/home/zzn/PycharmProjects/NAS_FPN/StateDicts/FPN_6-13_A.pkl"))
optimizer = torch.optim.Adam(net.parameters(), config['learning_rate'])
# optimizer = torch.optim.SGD(
#     net.parameters(), 1e-7, momentum=0.95, weight_decay=5e-4)

criterion = torch.nn.MSELoss(reduction='sum').cuda()
ae_batch = AEBatch().cuda()
se_batch = SEBatch().cuda()
modules = {
    'model': net,
    'shape': None,
    'ae': ae_batch,
    'se': se_batch,
    'loss': criterion
}

In [ ]:
step = 0
eval_loss = []
eval_mae = []
eval_rmse = []
for epoch_index in range(config['epoch']):
    dataset = train_dataset.shuffle()
    loss_list = []
    time_per_epoch = 0
    for train_img_index, train_img, train_gt in train_loader:
        if step % config['eval_per_step'] == 0:
            validate_MAE, validate_RMSE, validate_loss, time_cost = eval_model(
                config, eval_loader, modules, False)
            eval_loss.append(validate_loss)
            eval_mae.append(validate_MAE)
            eval_rmse.append(eval_rmse)
            sys.stdout.write(
                'In step {}, epoch {}, loss = {}, eval_mae = {}, eval_rmse = {}, time cost eval = {}s\n'
                .format(step, epoch_index, validate_loss, validate_MAE,
                        validate_RMSE, time_cost))
            sys.stdout.flush()
            #             save model
            if config['stage'] == 'numeration' and config[
                    'min_mae'] > validate_MAE:
                config['min_mae'] = validate_MAE
                torch.save(net.state_dict(), config['model_save_path'])

        net.train()
        torch.cuda.empty_cache()
        x = train_img
        y = train_gt
        start = time.time()
        prediction = net(x)
        loss = criterion(prediction, y)
        optimizer.zero_grad()
        loss.backward()
        loss_list.append(loss.data.item())
        optimizer.step()

        step += 1
        torch.cuda.synchronize()
        end = time.time()
        time_per_epoch += end - start

In step 0, epoch 0, loss = 59.6458178300124, eval_mae = 289.4889831542969, eval_rmse = 380.2806701660156, time cost eval = 35.83038020133972s
In step 300, epoch 1, loss = 42.4774145765619, eval_mae = 161.80984497070312, eval_rmse = 230.26812744140625, time cost eval = 35.52651524543762s
In step 600, epoch 2, loss = 38.784766742161345, eval_mae = 163.6050567626953, eval_rmse = 246.1707305908203, time cost eval = 35.40503787994385s
In step 900, epoch 3, loss = 35.46890822347704, eval_mae = 125.86602783203125, eval_rmse = 194.7615966796875, time cost eval = 35.32987570762634s
In step 1200, epoch 4, loss = 34.24294612433884, eval_mae = 145.8416290283203, eval_rmse = 210.3524932861328, time cost eval = 35.380532026290894s
In step 1500, epoch 5, loss = 34.40345111784044, eval_mae = 141.38861083984375, eval_rmse = 217.57847595214844, time cost eval = 35.39902186393738s
In step 1800, epoch 6, loss = 34.40743842753735, eval_mae = 169.80062866210938, eval_rmse = 249.60707092285156, time cost eva

In step 16800, epoch 56, loss = 26.098655660073835, eval_mae = 77.04352569580078, eval_rmse = 112.6972427368164, time cost eval = 34.67562198638916s
In step 17100, epoch 57, loss = 26.218149000471765, eval_mae = 91.3480224609375, eval_rmse = 125.14981079101562, time cost eval = 34.619566917419434s
In step 17400, epoch 58, loss = 26.299895961206037, eval_mae = 75.89817810058594, eval_rmse = 111.7504653930664, time cost eval = 34.68700098991394s
In step 17700, epoch 59, loss = 25.924794427641146, eval_mae = 72.67611694335938, eval_rmse = 110.78849029541016, time cost eval = 34.59959077835083s
In step 18000, epoch 60, loss = 26.027276445221116, eval_mae = 75.37609100341797, eval_rmse = 112.4930191040039, time cost eval = 34.67645883560181s
In step 18300, epoch 61, loss = 26.31022775697184, eval_mae = 89.9462661743164, eval_rmse = 121.63665771484375, time cost eval = 34.574082374572754s
In step 18600, epoch 62, loss = 25.83505193610768, eval_mae = 75.10618591308594, eval_rmse = 111.7787399

In step 33300, epoch 111, loss = 25.101916376051012, eval_mae = 69.4342041015625, eval_rmse = 108.7700424194336, time cost eval = 34.691051721572876s
In step 33600, epoch 112, loss = 25.383316664905337, eval_mae = 72.18258666992188, eval_rmse = 116.65374755859375, time cost eval = 34.618518352508545s
In step 33900, epoch 113, loss = 24.979572378672085, eval_mae = 72.54288482666016, eval_rmse = 112.81468963623047, time cost eval = 34.55200529098511s
In step 34200, epoch 114, loss = 25.350491373093572, eval_mae = 87.07207489013672, eval_rmse = 119.74461364746094, time cost eval = 34.70164132118225s
In step 34500, epoch 115, loss = 24.95082481614836, eval_mae = 78.4795913696289, eval_rmse = 109.8525390625, time cost eval = 34.64351558685303s
In step 34800, epoch 116, loss = 24.938587351159736, eval_mae = 71.215087890625, eval_rmse = 107.13567352294922, time cost eval = 34.60934138298035s
In step 35100, epoch 117, loss = 25.07770643915449, eval_mae = 70.85305786132812, eval_rmse = 111.0593

In [ ]:
import h5py
import scipy.io as scio
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
perspective_dir = "/home/zzn/Documents/Datasets/part_A_final/test_data/perspective_gt"
img_dir= "/home/zzn/Documents/Datasets/part_A_final/test_data/images"
count = 0
for i in range(182):
    perspective_map_name = '/IMG_' + str(i + 1) + ".mat"
    perspective_map = scio.loadmat(perspective_dir + perspective_map_name)['pmap'][:]
    if perspective_map[0][0] < 5:
        count += 1
        plt.imshow(Image.open(img_dir + '/IMG_' + str(i + 1) + ".jpg"))
        plt.show()